In [ ]:
import os
import yaml

In [ ]:
# basepath = '/workspace/nemo/vol/Corpus/RAVDESS'
# writepath = '/workspace/nemo/vol/corpus_flattened'
# for speaker in os.listdir(basepath):
#     for emotion in os.listdir(os.path.join(basepath, speaker)):
#         print(emotion)

In [ ]:
# writepath = '/workspace/nemo/vol/corpus_flattened'

# for speaker in os.listdir(writepath):
#     i = 0
#     for filename in os.listdir(os.path.join(writepath, speaker)):
#         new_filename = filename.replace("txt", "lab")
#         i = i+1
#         print(new_filename)
#         os.rename(os.path.join(writepath, speaker,filename), os.path.join(writepath, speaker, new_filename))
       
        
        
            
            

In [4]:
import torch

def explore_ckpt_file(ckpt_path):
    try:
        # Load the checkpoint data
        checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))

        # List all keys in the checkpoint dictionary (usually contains 'model_state_dict' and more)
        print("Keys in the checkpoint dictionary:\n")
        for key in checkpoint.keys():
            print(key)

        # Access the model's state_dict (modify 'model_state_dict' if different key)
        if 'model_state_dict' in checkpoint:
            state_dict = checkpoint['model_state_dict']

            # List all keys in the state_dict (these are the model's parameter names)
            print("\nKeys in the model's state_dict:\n")
            for key in state_dict.keys():
                print(key)

            # Access specific parameters (you can modify these to explore the data)
            print("\nExample: Accessing specific parameters:\n")
            parameter_name = "your_parameter_name_here"  # Change this to a specific parameter name
            if parameter_name in state_dict:
                parameter = state_dict[parameter_name]
                print(f"Parameter: {parameter_name}")
                print(f"Shape: {parameter.shape}")
                print(f"Data: {parameter}")
            else:
                print(f"Parameter {parameter_name} not found in the state_dict.")
        else:
            print("The 'model_state_dict' key not found in the checkpoint.")

    except Exception as e:
        print(f"Error: {str(e)}")

# if __name__ == "__main__":
    # Replace 'path/to/your/checkpoint.ckpt' with the actual path to your .ckpt file
#ckpt_file_path = '/workspace/nemo/vol/dataStore2/FastSpeech2/output/ckpt/EMOVDB_frozen/5500.pth.tar'
# ckpt_file_path = '/workspace/nemo/vol/dataStore2/FastSpeech2/output/ckpt/EMOVDB/500.pth.tar'
#explore_ckpt_file(ckpt_file_path)


In [5]:
ckpt_file_path = '/workspace/nemo/vol/FastSpeech2/output/ckpt/Nova/50000.pth.tar'
checkpoint = torch.load(ckpt_file_path, map_location=torch.device('cpu'))

model_config = yaml.load(open('/workspace/nemo/vol/FastSpeech2/config/Nova/model.yaml',
                              "r"), Loader=yaml.FullLoader)
preprocess_config = yaml.load(open('/workspace/nemo/vol/FastSpeech2/config/Nova/preprocess.yaml',
                              "r"), Loader=yaml.FullLoader)

In [6]:
checkpoint.keys()

dict_keys(['model', 'optimizer'])

In [7]:
checkpoint['model']['speaker_emb.weight'].size()

torch.Size([210, 512])

In [8]:
checkpoint['model']['emotion_emb.weight'].size()

torch.Size([13, 512])

In [9]:
from model import FastSpeech2

In [10]:
# import yaml
# model_config = yaml.load(open('/workspace/nemo/vol/dataStore2/FastSpeech2/config/emovdb_frozen/model.yaml',
#                               "r"), Loader=yaml.FullLoader)
# preprocess_config = yaml.load(open('/workspace/nemo/vol/dataStore2/FastSpeech2/config/emovdb_frozen/preprocess.yaml',
#                               "r"), Loader=yaml.FullLoader)

In [11]:
fp = FastSpeech2(model_config=model_config, preprocess_config=preprocess_config)

In [12]:
import numpy as np
import torch
from synthesize import preprocess_english
text = """I fear that you have lost so much already that you cannot bare to lose more. In a world without Gold, we might have been heroes.
Our creed does not command us to be free, it commands us to be wise."""
speakers = torch.tensor([163]).to('cuda')
emotion = torch.tensor([2]).to('cuda')
ids = raw_texts = text

texts = torch.tensor([preprocess_english(text, preprocess_config)]).to('cuda')
text_lens = torch.tensor([len(texts[0])]).to('cuda')
batchs = [(ids, raw_texts, speakers, emotion, texts, text_lens, max(text_lens))]

/tmp/ipykernel_3890811/4134923763.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  texts = torch.tensor([preprocess_english(text, preprocess_config)]).to('cuda')


In [13]:
fp.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [14]:
fp = fp.to('cuda')

In [15]:
%time predictions = fp(*(batchs[0][2:]))

CPU times: user 746 ms, sys: 373 ms, total: 1.12 s
Wall time: 1.17 s


In [16]:
from utils.model import vocoder_infer, get_vocoder
from scipy.io import wavfile
import os

%time mel_predictions = predictions[1].transpose(1, 2)

CPU times: user 23 µs, sys: 10 µs, total: 33 µs
Wall time: 42.2 µs


In [17]:
# import torchaudio
# from speechbrain.pretrained import HIFIGAN
# from speechbrain.lobes.models.FastSpeech2 import mel_spectogram

# # Load a pretrained HIFIGAN Vocoder
# hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-libritts-22050Hz", savedir="tmpdir_voc22050")

# # Load an audio file (an example file can be found in this repository)
# # Ensure that the audio signal is sampled at 22050 Hz; refer to the provided link for a 16000 Hz Vocoder.
# #signal, rate = torchaudio.load('speechbrain/tts-hifigan-libritts-22050H/example_22kHz.wav')
# #signal, rate = torchaudio.load('/home/mirco/Downloads/example_22kHz.wav')

# # Ensure the audio is sigle channel
# #signal = signal[0].squeeze()

# #torchaudio.save('waveform.wav', signal.unsqueeze(0), 22050)

# # Compute the mel spectrogram.
# # IMPORTANT: Use these specific parameters to match the Vocoder's training settings for optimal results.
# # spectrogram, _ = mel_spectogram(
# #     audio=signal.squeeze(),
# #     sample_rate=22050,
# #     hop_length=256,
# #     win_length=1024,
# #     n_mels=80,
# #     n_fft=1024,
# #     f_min=0.0,
# #     f_max=8000.0,
# #     power=1,
# #     normalized=False,
# #     min_max_energy_norm=True,
# #     norm="slaney",
# #     mel_scale="slaney",
# #     compression=True
# # )

# # Convert the spectrogram to waveform
# waveforms = hifi_gan.decode_batch(mel_predictions)

# # Save the reconstructed audio as a waveform
# torchaudio.save('hg_22.05_libritts_sample.wav', waveforms.squeeze(1), 22050)

# # If everything is set up correctly, the original and reconstructed audio should be nearly indistinguishable.


In [18]:
# import IPython.display as ipd
# ipd.display(ipd.Audio(waveforms.squeeze(1), rate=22050))

In [19]:
#%time mel_predictions = predictions[1].transpose(1, 2)

In [20]:
# from nemo.collections.tts.models import HifiGanModel
# vocoder = HifiGanModel.load_from_checkpoint("/workspace/nemo/vol/FastSpeech2/hifigan_ft/HifiGan/2023-07-28_02-15-51/checkpoints/HifiGan--val_loss=0.3318-epoch=944.ckpt")
# vocoder = vocoder.eval().cuda()

In [21]:
from nemo.collections.tts.models import HifiGanModel
vocoder = HifiGanModel.from_pretrained("tts_en_hifigan")
vocoder = vocoder.eval().cuda()

NOTE! Installing ujson may make loading annotations faster.


[NeMo W 2023-11-27 14:05:05 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-27 14:05:06 experimental:27] Module <class 'nemo.collections.tts.models.fastpitch_ssl.FastPitchModel_SSL'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-27 14:05:06 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-27 14:05:06 experimental:27] Module <class 'nemo.collections.tts.models.radtts.RadTTSModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-11-27 14:05:06 experimental:27] Module <class 'nemo.collections.tts.models.ssl_tts.SSLDisentangler'> is expe

[NeMo I 2023-11-27 14:05:06 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2023-11-27 14:05:06 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2023-11-27 14:05:06 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-11-27 14:05:09 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-11-27 14:05:09 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-11-27 14:05:09 features:291] PADDING: 0


[NeMo W 2023-11-27 14:05:09 features:268] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-11-27 14:05:09 features:291] PADDING: 0
[NeMo I 2023-11-27 14:05:11 save_restore_connector:249] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [22]:
audio = vocoder.convert_spectrogram_to_audio(spec=mel_predictions)
audio.cpu()

tensor([[ 0.0767,  0.0161, -0.0865,  ..., -0.0087,  0.0021,  0.0027]],
       grad_fn=<ToCopyBackward0>)

In [23]:
import IPython.display as ipd
ipd.display(ipd.Audio(audio.cpu().detach(), rate=22050))

In [33]:
import torchaudio
torchaudio.save("josh_amused.wav", audio.cpu().detach(), 22050, format='wav')

In [34]:
# from utils.model import vocoder_infer, get_vocoder
# from scipy.io import wavfile
# import os

# mel_predictions = predictions[1].transpose(1, 2)
# vocoder = get_vocoder(model_config, 'cuda')
# lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
# wav_predictions = vocoder_infer(
#     mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
# )

# sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
# outfilename = "josh_disgusted"
# for wav, basename in zip(wav_predictions, [outfilename]):
#     wavfile.write(os.path.join( "{}.wav".format(basename)), sampling_rate, wav)

In [35]:
with open("/workspace/nemo/vol/FastSpeech2/output/log/RAVDESS/train/log.txt", "r") as f:
    logs = f.read()

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/nemo/vol/FastSpeech2/output/log/RAVDESS/train/log.txt'

In [46]:
logs_ = logs.split("\n")

In [57]:
item = logs_[0]
key_value_pairs = [item.split(':') for item in text.split(', ')[1:]]

# Convert the key-value pairs into a dictionary
result_dict = {key.strip(): float(value) for key, value in key_value_pairs}


In [116]:
logs_[1]

'Step 200/900000, Total Loss: 7.2434, Mel Loss: 2.5218, Mel PostNet Loss: 2.0830, Pitch Loss: 0.5603, Energy Loss: 1.5156, Duration Loss: 0.5626'

In [70]:
loss_dict = {}
for i, log_ in enumerate(logs_):
    try:
        vals = [i.split(":")[1] for i in log_.replace("Step", "Step:").split(",")]
        iter_dict = {}
        iter_dict['iter'] = vals[0]
        iter_dict['total_loss'] = vals[1]
        iter_dict['mel_loss'] = vals[2]
        iter_dict['postnet_loss'] = vals[3]
        iter_dict['pitch_loss'] = vals[4]
        iter_dict['energy_loss'] = vals[5]
        iter_dict['dur_loss'] = vals[6]
        loss_dict[(i+1)*100] = iter_dict
    except:
        continue
    

In [117]:
len(logs_)

9011

In [128]:
loss_dict1 = {}
for i, log_ in enumerate(logs_):
    try:
        vals = [j.split(":")[1] for j in log_.replace("Step", "Step:").split(",")]
#         print(vals[0])
#         iter_dict = {}
#         iter_dict['iter'] = vals[0]
#         iter_dict['total_loss'] = vals[1]
#         iter_dict['mel_loss'] = vals[2]
#         iter_dict['postnet_loss'] = vals[3]
#         iter_dict['pitch_loss'] = vals[4]
#         iter_dict['energy_loss'] = vals[5]
#         iter_dict['dur_loss'] = vals[6]
        if int(vals[0].split('/')[0]) % 100000 == 0:
            loss_dict1[vals[0].split("/")[0]] = vals[1]
    except:
        continue

In [113]:
vals[]

[' 900000/900000',
 ' 0.9803',
 ' 0.4343',
 ' 0.4329',
 ' 0.0312',
 ' 0.0499',
 ' 0.0321']

In [114]:
int(vals[0].split('/')[0])

900000

In [115]:
vals[0]

' 900000/900000'

In [93]:
52%10

2

In [129]:
loss_dict1

{' 100000': ' 1.1884',
 ' 200000': ' 1.0851',
 ' 300000': ' 1.1187',
 ' 400000': ' 0.9975',
 ' 500000': ' 0.9179',
 ' 600000': ' 1.0297',
 ' 700000': ' 0.9462',
 ' 800000': ' 0.9281',
 ' 900000': ' 0.9803'}

In [85]:
min_value = min(loss_dict1.values())

In [86]:
min_value

' 0.8560'

In [47]:
logs_[0]

'Step 100/900000, Total Loss: 10.8035, Mel Loss: 3.7052, Mel PostNet Loss: 3.1430, Pitch Loss: 1.5909, Energy Loss: 1.4730, Duration Loss: 0.8915'

In [ ]:
logs_dict = {"iterations": i, "total_loss": }

In [ ]:
iterations, total_loss, mel_loss, postnet_loss, pitch_loss, energy_loss, dur_loss = []

In [21]:
#checkpoint['model'].keys()

In [3]:
# vocoder_dict = torch.load("/workspace/nemo/vol/TTS_voices/FP_checkpoints/HifiGan--val_loss=0.3257-epoch=9479.ckpt")

In [28]:
#vocoder_dict

In [2]:
# import json
# import hifigan
# with open("hifigan/config.json", "r") as f:
#             config = json.load(f)
# config = hifigan.AttrDict(config)
# vocoder = hifigan.Generator(config)

In [3]:
from nemo.collections.tts.models import HifiGanModel
vocoder = HifiGanModel.from_pretrained("tts_en_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2023-11-20 06:29:34 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2023-11-20 06:29:34 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2023-11-20 06:29:34 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-11-20 06:29:39 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-11-20 06:29:39 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-11-20 06:29:39 features:291] PADDING: 0


[NeMo W 2023-11-20 06:29:39 features:268] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-11-20 06:29:39 features:291] PADDING: 0
[NeMo I 2023-11-20 06:29:41 save_restore_connector:249] Model HifiGanModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.19.0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [ ]:
text = [
        "My name is Samrat and I love stock market trading, Intraday! Intraday is the way to go baby!",
        "Trying out a second statement with you !", 
        "Trying out the third and final statement.",
        "Just kidding , this is the final statement."
]

speakers = torch.tensor([0,1,2,3]).to('cuda')
emotion = torch.tensor([0,1,2,3]).to('cuda')

text = [preprocess_english(i, preprocess_config) for i in text]
text = pad_1D(text)
texts = torch.tensor(text).to('cuda')

text_lens = torch.tensor([len(i) for i in texts]).to('cuda')

batchs = [(speakers, emotion, texts, text_lens, max(text_lens))]
%time predictions = fp(*(batchs[0]), 
                        p_control=pitch_control,
                        e_control=energy_control,
                        d_control=duration_control)
mel_predictions = predictions[1].permute(0, 2, 1)

In [25]:
import numpy as np

mel_path = '/workspace/nemo/vol/extvol2/mels/mel_2086_8_Ses04F_script01_3_M040.npy'
mel_predictions = np.load(mel_path)
mel_predictions.shape

(80, 490)

In [27]:
a = np.expand_dims(mel_predictions, axis = 0)
a.shape

(1, 80, 490)

In [32]:
mel_predictions = torch.Tensor(a)
mel_predictions = mel_predictions.to('cuda')

In [35]:
audio = vocoder.convert_spectrogram_to_audio(spec=mel_predictions)
audio.cpu()

for i in audio:
    ipd.display(ipd.Audio(i.cpu().detach(), rate=22050))

In [34]:
import IPython.display as ipd